
Graded Challenge 6

Nama  : Mirza Rendra Sjarief
Batch : CODA-RMT-001

Program ini dibuat untuk melakukan ETL (ekstrak, transform, dan load). Proses ekstrak dilakukan dengan mengambil data dari bigquery, kemudian Load file-file csv tersebut ke PySpark dataframe. Dilanjutkan dengan proses transform dengan library pyspark Proses cleaning dan transform dapat menerapkan perubahan nama kolom, cek dan hapus missing values, menambah atau menghapus kolom. Kemudian data di load  dan dihubungkan ke database PostgreSQL.


In [1]:
#import nececary librabry
import pyspark
import findspark
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, sum, concat, lit, when, to_date
from sqlalchemy import create_engine



In [2]:
sc = pyspark.SparkContext()
sc

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/12/05 17:54:42 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


<SparkContext master=local[*] appName=pyspark-shell>

- Objek konfigurasi untuk Spark, seperti nama aplikasi dan pengaturan master juga membaca data dari file dan membuat RDD.

In [3]:

spark = SparkSession.builder.getOrCreate()
spark

- Fungsi Utama SparkSession.builder.getOrCreate() Membuat SparkSession Baru, Menyediakan builder untuk mengonfigurasi SparkSession.
Menetapkan nama aplikasi Spark. Menetapkan mode Spark . getOrCreate(): Memastikan bahwa hanya ada satu SparkSession, baik itu baru atau yang sudah ada.



## PySpark Data Transformation

- Data transform dan cleaning tabel entity users

In [4]:
df_users = spark.read.csv('/Users/mac/Documents/Hacktiv8/GC6/p2-coda001-rmt-gc6-mirzasjarief/look_users.csv', header=True, inferSchema=True)
df_users.show()

+-----+-----------+---------+--------------------+---+------+-----+--------------------+-----------+--------+-------+------------+------------+--------------+--------------------+--------------------+
|   id| first_name|last_name|               email|age|gender|state|      street_address|postal_code|    city|country|    latitude|   longitude|traffic_source|          created_at|           user_geom|
+-----+-----------+---------+--------------------+---+------+-----+--------------------+-----------+--------+-------+------------+------------+--------------+--------------------+--------------------+
|70913|       Levi|    Hogan|levihogan@example...| 23|     M| Acre|89243 Jennifer Ga...|  69980-000|    null| Brasil|-8.065346116|-72.87094866|      Facebook| 2021-06-16 20:46:00|POINT(-72.8709486...|
|34003|Christopher|   Obrien|christopherobrien...| 65|     M| Acre|367 David Isle Ap...|  69980-000|    null| Brasil|-8.065346116|-72.87094866|       Organic| 2021-12-01 15:59:00|POINT(-72.8709486

- Load file entity users dari bigquery menggunakan metode spark.read()


In [5]:
df_users = df_users.dropDuplicates()


- Code ini memastikan tidak ada duplikat data yang dapat membuat bias pada analisis dengan metode .dropDuplicates()

In [6]:
df_users.printSchema()

root
 |-- id: integer (nullable = true)
 |-- first_name: string (nullable = true)
 |-- last_name: string (nullable = true)
 |-- email: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- gender: string (nullable = true)
 |-- state: string (nullable = true)
 |-- street_address: string (nullable = true)
 |-- postal_code: string (nullable = true)
 |-- city: string (nullable = true)
 |-- country: string (nullable = true)
 |-- latitude: double (nullable = true)
 |-- longitude: double (nullable = true)
 |-- traffic_source: string (nullable = true)
 |-- created_at: timestamp (nullable = true)
 |-- user_geom: string (nullable = true)



- Melihat schema dari table entity users, nullable status tiap kolom dan data type dengan hasil tipe data String = 11, Int = 2, double/float = 2, Timestamp = 1

In [7]:
df_users.describe().show()

24/12/05 13:36:56 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+-------+-----------------+----------+---------+--------------------+------------------+------+---------------+--------------------+------------------+-----+-------------+------------------+------------------+--------------+--------------------+
|summary|               id|first_name|last_name|               email|               age|gender|          state|      street_address|       postal_code| city|      country|          latitude|         longitude|traffic_source|           user_geom|
+-------+-----------------+----------+---------+--------------------+------------------+------+---------------+--------------------+------------------+-----+-------------+------------------+------------------+--------------+--------------------+
|  count|            39184|     39184|    39184|               39184|             39184| 39184|          39184|               39184|             39184|39184|        39184|             39184|             39184|         39184|               39184|
|   mean|49823.1

- Melihat total isi data di tiap kolom entity users, dan mean, median, std dev, max dan min data integer

In [7]:


null_data_users = df_users.select([sum(col(c).isNull().cast("int")).alias(c) for c in df_users.columns])
null_data_users.show()

+---+----------+---------+-----+---+------+-----+--------------+-----------+----+-------+--------+---------+--------------+----------+---------+
| id|first_name|last_name|email|age|gender|state|street_address|postal_code|city|country|latitude|longitude|traffic_source|created_at|user_geom|
+---+----------+---------+-----+---+------+-----+--------------+-----------+----+-------+--------+---------+--------------+----------+---------+
|  0|         0|        0|    0|  0|     0|    0|             0|          0|   0|      0|       0|        0|             0|         0|        0|
+---+----------+---------+-----+---+------+-----+--------------+-----------+----+-------+--------+---------+--------------+----------+---------+



- code ini memiliki perintah untuk mengecek mising value pada dataset

In [9]:
df_users = df_users.drop("latitude","longitude","created_at")

df_users.show()

+-----+-----------+---------+--------------------+---+------+---------+--------------------+-----------+-----------+-------------+--------------+--------------------+
|   id| first_name|last_name|               email|age|gender|    state|      street_address|postal_code|       city|      country|traffic_source|           user_geom|
+-----+-----------+---------+--------------------+---+------+---------+--------------------+-----------+-----------+-------------+--------------+--------------------+
|98265|     Cheryl| Mcintosh|cherylmcintosh@ex...| 67|     F|    Aichi|82272 Hale Villag...|   446-0001|  Anjo city|        Japan|        Search|POINT(137.0968981...|
|61804|    Matthew|     Wood|matthewwood@examp...| 21|     M|  Alagoas|432 Collins Exten...|  57230-000|   Coruripe|       Brasil|        Search|POINT(-36.2327050...|
|52949|   Courtney|  Kennedy|courtneykennedy@e...| 63|     F|    Amapá|  694 Rachel Heights|  68925-000|    Santana|       Brasil|         Email|POINT(-51.3775319...

- Berikut adalah menghilangkan table latitude dan longitude, tidak diperlukan  karena sudah didefinisikan oleh user_geom, juga kolom created add yang sudah tidak diperlukan karena keterangan user sudah cukup lengkap

### Data transform dan cleaning tabel entity products

In [10]:
df_products = spark.read.csv('/Users/mac/Documents/Hacktiv8/GC6/p2-coda001-rmt-gc6-mirzasjarief/look_products.csv', header=True, inferSchema=True)
df_products.show()

+-----+------------------+-----------+--------------------+-----+------------------+----------+--------------------+----------------------+
|   id|              cost|   category|                name|brand|      retail_price|department|                 sku|distribution_center_id|
+-----+------------------+-----------+--------------------+-----+------------------+----------+--------------------+----------------------+
|13842| 2.518749990849756|Accessories|Low Profile Dyed ...|   MG|              6.25|     Women|EBD58B8A3F1D72F42...|                     1|
|13928|2.3383499148894105|Accessories|Low Profile Dyed ...|   MG| 5.949999809265137|     Women|2EAC42424D12436BD...|                     1|
|14115| 4.879559879379869|Accessories|Enzyme Regular So...|   MG|10.989999771118164|     Women|EE364229B2791D1EF...|                     1|
|14157| 4.648769887297898|Accessories|Enzyme Regular So...|   MG|10.989999771118164|     Women|00BD13095D06C20B1...|                     1|
|14273| 6.5079298864

- Load file entity products dari bigquery menggunakan metode spark.read()


In [11]:
df_products = df_products.dropDuplicates()


- Code ini memastika tidak ada data duplikat pada dataset


In [12]:
df_products.printSchema()

root
 |-- id: integer (nullable = true)
 |-- cost: double (nullable = true)
 |-- category: string (nullable = true)
 |-- name: string (nullable = true)
 |-- brand: string (nullable = true)
 |-- retail_price: double (nullable = true)
 |-- department: string (nullable = true)
 |-- sku: string (nullable = true)
 |-- distribution_center_id: integer (nullable = true)



- Melihat schema dari table entity products, nullable status tiap kolom dan data type dengan hasil tipe data String = 5, Int = 2, double/float = 2, Timestamp = 1

In [13]:
df_products.describe().show(vertical=True)

-RECORD 0--------------------------------------
 summary                | count                
 id                     | 29120                
 cost                   | 29120                
 category               | 29120                
 name                   | 29118                
 brand                  | 29096                
 retail_price           | 29120                
 department             | 29120                
 sku                    | 29120                
 distribution_center_id | 29120                
-RECORD 1--------------------------------------
 summary                | mean                 
 id                     | 14560.5              
 cost                   | 28.481774240436966   
 category               | NULL                 
 name                   | NULL                 
 brand                  | Infinity             
 retail_price           | 59.220163865731955   
 department             | NULL                 
 sku                    | NULL          

- Melihat total isi data di tiap kolom entity products, serta mean, median, std dev, max dan min data integer. Melalui info diatas ditemukan missing value pada kolom brand (24) dan name (2).

In [14]:
df_products = df_products.na.fill({"brand": "Unknown", "name": "Unknown"})
df_products.show()

+-----+------------------+--------------------+--------------------+-----------------+------------------+----------+--------------------+----------------------+
|   id|              cost|            category|                name|            brand|      retail_price|department|                 sku|distribution_center_id|
+-----+------------------+--------------------+--------------------+-----------------+------------------+----------+--------------------+----------------------+
| 8162| 9.820219895048735|               Suits|ICE smart stretch...|              Ice|16.989999771118164|     Women|7F2776F553FE2D5F8...|                     1|
|13312| 30.31200013216585|                Swim|TYR Sport Women's...|              TYR|              72.0|     Women|9DD48D8DF816E1B2B...|                     1|
|  349| 16.98137974072516|         Tops & Tees|Patty Women Cute ...|            Patty|31.979999542236328|     Women|0BB4AEC1710521C12...|                     1|
|  820|15.732739771852494|        

- Code berikut adalah metode mengisi missing value na.fill(), dalam kasus ini tidak perlu menghapus data karena missing value tidak terlalu banyak maka dapat diganti dengan keterangan lain, dan agar tidak mengubah esensi data.

In [15]:

null_data_products = df_products.select([sum(col(c).isNull().cast("int")).alias(c) for c in df_products.columns])
null_data_products.show()

+---+----+--------+----+-----+------------+----------+---+----------------------+
| id|cost|category|name|brand|retail_price|department|sku|distribution_center_id|
+---+----+--------+----+-----+------------+----------+---+----------------------+
|  0|   0|       0|   0|    0|           0|         0|  0|                     0|
+---+----+--------+----+-----+------------+----------+---+----------------------+



- Code berikut menunjukan dengan spesifik bahwa tidak ada lagi kolom dengan missing value

In [16]:
df_products = df_products.withColumnRenamed("cost", "product_cost").withColumnRenamed("category", "product_category").withColumnRenamed("name", "product_name").withColumnRenamed("brand", "product_brand")


df_products.show(5)

+-----+------------------+----------------+--------------------+-------------+------------------+----------+--------------------+----------------------+
|   id|      product_cost|product_category|        product_name|product_brand|      retail_price|department|                 sku|distribution_center_id|
+-----+------------------+----------------+--------------------+-------------+------------------+----------+--------------------+----------------------+
| 8162| 9.820219895048735|           Suits|ICE smart stretch...|          Ice|16.989999771118164|     Women|7F2776F553FE2D5F8...|                     1|
|13312| 30.31200013216585|            Swim|TYR Sport Women's...|          TYR|              72.0|     Women|9DD48D8DF816E1B2B...|                     1|
|  349| 16.98137974072516|     Tops & Tees|Patty Women Cute ...|        Patty|31.979999542236328|     Women|0BB4AEC1710521C12...|                     1|
|  820|15.732739771852494|        Sweaters|Patty Women Comfy...|        Patty| 33.

- Code berikut merubah nama kolom untuk mempermudah identifikasi

### Data transform dan cleaning tabel entity orders

In [17]:
df_orders = spark.read.csv('/Users/mac/Documents/Hacktiv8/GC6/p2-coda001-rmt-gc6-mirzasjarief/bq-result_look_orders.csv', header=True, inferSchema=True)
df_orders.show()

+--------+-------+---------+------+-------------------+-----------+----------+------------+-----------+
|order_id|user_id|   status|gender|         created_at|returned_at|shipped_at|delivered_at|num_of_item|
+--------+-------+---------+------+-------------------+-----------+----------+------------+-----------+
|       4|      3|Cancelled|     F|2024-04-05 16:51:00|       NULL|      NULL|        NULL|          4|
|      36|     28|Cancelled|     F|2022-07-09 17:00:00|       NULL|      NULL|        NULL|          1|
|      42|     34|Cancelled|     F|2024-07-01 20:05:00|       NULL|      NULL|        NULL|          1|
|      68|     50|Cancelled|     F|2024-11-08 00:27:00|       NULL|      NULL|        NULL|          2|
|      69|     50|Cancelled|     F|2024-02-24 00:27:00|       NULL|      NULL|        NULL|          1|
|      85|     63|Cancelled|     F|2023-07-28 14:53:00|       NULL|      NULL|        NULL|          1|
|      91|     66|Cancelled|     F|2023-08-14 00:20:00|       NU

In [18]:
df_orders = df_orders.dropDuplicates()


- Code ini memastikan tidak ada duplikat pada tabel yang dapat menyebabkan bias pada data

In [19]:
df_orders.printSchema()

root
 |-- order_id: integer (nullable = true)
 |-- user_id: integer (nullable = true)
 |-- status: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- created_at: timestamp (nullable = true)
 |-- returned_at: timestamp (nullable = true)
 |-- shipped_at: timestamp (nullable = true)
 |-- delivered_at: timestamp (nullable = true)
 |-- num_of_item: integer (nullable = true)



- Melihat schema dari table entity Orders, nullable status tiap kolom dan data type dengan hasil tipe data String = 2, Int = 3, Timestamp = 4

In [20]:
df_orders.describe().show(vertical=True)

-RECORD 0-------------------------
 summary     | count              
 order_id    | 125905             
 user_id     | 125905             
 status      | 125905             
 gender      | 125905             
 num_of_item | 125905             
-RECORD 1-------------------------
 summary     | mean               
 order_id    | 62953.0            
 user_id     | 50073.943306461224 
 status      | NULL               
 gender      | NULL               
 num_of_item | 1.4527222906159405 
-RECORD 2-------------------------
 summary     | stddev             
 order_id    | 36345.787158440624 
 user_id     | 28871.860405565214 
 status      | NULL               
 gender      | NULL               
 num_of_item | 0.8073639535699144 
-RECORD 3-------------------------
 summary     | min                
 order_id    | 1                  
 user_id     | 1                  
 status      | Cancelled          
 gender      | F                  
 num_of_item | 1                  
-RECORD 4-----------

In [21]:

null_data_orders = df_orders.select([sum(col(c).isNull().cast("int")).alias(c) for c in df_orders.columns])
null_data_orders.show()

+--------+-------+------+------+----------+-----------+----------+------------+-----------+
|order_id|user_id|status|gender|created_at|returned_at|shipped_at|delivered_at|num_of_item|
+--------+-------+------+------+----------+-----------+----------+------------+-----------+
|       0|      0|     0|     0|         0|     113337|     44179|       81852|          0|
+--------+-------+------+------+----------+-----------+----------+------------+-----------+



- Code berikut menunjukan total missing value pada tabel orders dimana pada kolom returned_at, delivered_at, shipped_at memiliki missing value lebih dari 50%  dari total data  yang ada namun tidak saya hapus karen missing value tersebut memiliki  arti status cancelled sehingga masih dibutuhkan dalam analisis jangka panjang.

- Berikut code dan metode untuk menghapus kolom .drop(), dimana kita menghapus kolom returned_at,shipped_at,delivered_at

In [22]:
df_orders = df_orders.withColumnRenamed("order_id", "id").withColumnRenamed("num_of_item", "order_quantity")


df_orders.show(5)

+-----+-------+--------+------+-------------------+-------------------+-------------------+-------------------+--------------+
|   id|user_id|  status|gender|         created_at|        returned_at|         shipped_at|       delivered_at|order_quantity|
+-----+-------+--------+------+-------------------+-------------------+-------------------+-------------------+--------------+
| 8723|   6933|Returned|     F|2024-08-28 13:44:00|2024-09-04 10:23:00|2024-08-29 19:36:00|2024-09-02 05:27:00|             1|
|18656|  14900|Returned|     F|2019-03-19 14:23:00|2019-03-25 17:12:00|2019-03-21 09:54:00|2019-03-22 22:36:00|             1|
|20291|  16180|Returned|     F|2023-01-19 01:14:00|2023-01-25 01:32:00|2023-01-19 18:56:00|2023-01-23 08:52:00|             1|
|22052|  17573|Returned|     F|2023-11-18 01:15:00|2023-11-20 16:47:00|2023-11-18 12:34:00|2023-11-18 19:53:00|             1|
|28139|  22432|Returned|     F|2019-08-13 14:06:00|2019-08-19 16:29:00|2019-08-16 00:56:00|2019-08-17 15:12:00|

- Code ini berfungsi untuk mengubah nama kolom dengan metode withColumnRenamed(), dalam kasus ini mengubah kolom order_id dan num_of_item, untuk memperjelas definisinya masing-masing

In [23]:
df_orders = df_orders.drop("gender")
df_orders.show()

+-----+-------+--------+-------------------+-------------------+-------------------+-------------------+--------------+
|   id|user_id|  status|         created_at|        returned_at|         shipped_at|       delivered_at|order_quantity|
+-----+-------+--------+-------------------+-------------------+-------------------+-------------------+--------------+
| 8723|   6933|Returned|2024-08-28 13:44:00|2024-09-04 10:23:00|2024-08-29 19:36:00|2024-09-02 05:27:00|             1|
|18656|  14900|Returned|2019-03-19 14:23:00|2019-03-25 17:12:00|2019-03-21 09:54:00|2019-03-22 22:36:00|             1|
|20291|  16180|Returned|2023-01-19 01:14:00|2023-01-25 01:32:00|2023-01-19 18:56:00|2023-01-23 08:52:00|             1|
|22052|  17573|Returned|2023-11-18 01:15:00|2023-11-20 16:47:00|2023-11-18 12:34:00|2023-11-18 19:53:00|             1|
|28139|  22432|Returned|2019-08-13 14:06:00|2019-08-19 16:29:00|2019-08-16 00:56:00|2019-08-17 15:12:00|             1|
|31428|  25094|Returned|2022-12-13 11:04

- Berikut ini menghapus redundant kolom gender yang sudah dimiliki oleh kolom table users.

In [24]:
df_orders_date = df_orders.withColumn('created_at', to_date(col('created_at')))

- merubah tipe data ke date 

### Data transform dan cleaning tabel entity Order_Items

In [25]:
df_order_items = spark.read.csv('/Users/mac/Documents/Hacktiv8/GC6/p2-coda001-rmt-gc6-mirzasjarief/look_order_items.csv', header=True, inferSchema=True)
df_order_items.show()

+------+--------+-------+----------+-----------------+----------+--------------------+--------------------+-------------------+-------------------+------------------+
|    id|order_id|user_id|product_id|inventory_item_id|    status|          created_at|          shipped_at|       delivered_at|        returned_at|        sale_price|
+------+--------+-------+----------+-----------------+----------+--------------------+--------------------+-------------------+-------------------+------------------+
| 21563|   14879|  11850|     14235|            58379| Cancelled| 2023-03-26 16:07:53|                NULL|               NULL|               NULL|0.0199999995529651|
|136838|   94494|  75766|     14235|           369696| Cancelled|2024-12-02 17:16:...|                NULL|               NULL|               NULL|0.0199999995529651|
|118204|   81671|  65373|     14235|           319399|  Complete| 2023-05-24 18:09:45| 2023-05-25 09:45:00|2023-05-28 17:37:00|               NULL|0.0199999995529651

In [26]:
df_order_items = df_order_items.dropDuplicates()


- Code ini memastikan tidak ada duplikat pada tabel yang dapat menyebabkan bias pada data

In [27]:
df_order_items.printSchema()

root
 |-- id: integer (nullable = true)
 |-- order_id: integer (nullable = true)
 |-- user_id: integer (nullable = true)
 |-- product_id: integer (nullable = true)
 |-- inventory_item_id: integer (nullable = true)
 |-- status: string (nullable = true)
 |-- created_at: timestamp (nullable = true)
 |-- shipped_at: timestamp (nullable = true)
 |-- delivered_at: timestamp (nullable = true)
 |-- returned_at: timestamp (nullable = true)
 |-- sale_price: double (nullable = true)



- Melihat schema dari table entity Orders, nullable status tiap kolom dan data type dengan hasil tipe data String = 1, Int = 6, Timestamp = 4, double = 1

In [28]:
df_order_items.describe().show()

+-------+-----------------+-----------------+------------------+------------------+------------------+---------+------------------+
|summary|               id|         order_id|           user_id|        product_id| inventory_item_id|   status|        sale_price|
+-------+-----------------+-----------------+------------------+------------------+------------------+---------+------------------+
|  count|            63936|            63936|             63936|             63936|             63936|    63936|             63936|
|   mean|90295.94263013013|62335.82133696196| 49874.88289852352|15255.627674549549| 243993.4831081081|     NULL|18.716753274334028|
| stddev|52217.25302045862|36052.59242782816|28941.562710163096| 8650.946749518132|141060.15392633106|     NULL| 6.712214229049677|
|    min|                3|                3|                 1|                18|                 8|Cancelled|0.0199999995529651|
|    max|           180678|           124760|             99999|            

In [29]:
null_data_order_items = df_order_items.select([sum(col(c).isNull().cast("int")).alias(c) for c in df_order_items.columns])
null_data_order_items.show()


+---+--------+-------+----------+-----------------+------+----------+----------+------------+-----------+----------+
| id|order_id|user_id|product_id|inventory_item_id|status|created_at|shipped_at|delivered_at|returned_at|sale_price|
+---+--------+-------+----------+-----------------+------+----------+----------+------------+-----------+----------+
|  0|       0|      0|         0|                0|     0|         0|     22377|       41686|      57770|         0|
+---+--------+-------+----------+-----------------+------+----------+----------+------------+-----------+----------+



- Kali ini karena missing value yang lebih dari 50% dan sudah ada table mengenai shipped_at,delivered_at,returned_at pada table orders maka selanjutnya akan dihapus seluruh data yang memiliki missing value

In [30]:
df_order_items_date = df_order_items.withColumn('created_at', to_date(col('created_at'))).withColumn('shipped_at', to_date(col('shipped_at'))).withColumn('delivered_at', to_date(col('delivered_at')))

- berikut adalah merubah data type timestamp menjadi date


### Data transform dan cleaning tabel entity Inventory Items

In [31]:
df_inv = spark.read.csv('/Users/mac/Documents/Hacktiv8/GC6/p2-coda001-rmt-gc6-mirzasjarief/look_inventory_items.csv', header=True, inferSchema=True)
df_inv.show()

+------+----------+-------------------+-------------------+------------------+----------------+--------------------+------------------+--------------------+------------------+--------------------+------------------------------+
|    id|product_id|         created_at|            sold_at|              cost|product_category|        product_name|     product_brand|product_retail_price|product_department|         product_sku|product_distribution_center_id|
+------+----------+-------------------+-------------------+------------------+----------------+--------------------+------------------+--------------------+------------------+--------------------+------------------------------+
| 19364|     13844|2024-02-10 00:00:05|2024-04-05 16:07:05| 2.768039897618853|     Accessories|(ONE) 1 Satin Hea...|Funny Girl Designs|   6.989999771118164|             Women|2A3E953A5E3D81E67...|                             7|
| 19365|     13844|2023-05-31 23:56:00|               NULL| 2.768039897618853|     Acces

In [32]:
df_inv = df_inv.dropDuplicates()


- Berikut menghapus duplikat data pada tabel inventory_items

In [33]:
df_inv.printSchema()

root
 |-- id: integer (nullable = true)
 |-- product_id: integer (nullable = true)
 |-- created_at: timestamp (nullable = true)
 |-- sold_at: timestamp (nullable = true)
 |-- cost: double (nullable = true)
 |-- product_category: string (nullable = true)
 |-- product_name: string (nullable = true)
 |-- product_brand: string (nullable = true)
 |-- product_retail_price: double (nullable = true)
 |-- product_department: string (nullable = true)
 |-- product_sku: string (nullable = true)
 |-- product_distribution_center_id: integer (nullable = true)



- Metode ini digunakan untuk melihat status nullable dan data type table inventory_items yang memiliki data string = 5, integer = 3, timestamp = 2, double = 2

In [34]:
null_inv = df_inv.select([sum(col(c).isNull().cast("int")).alias(c) for c in df_inv.columns])
null_inv.show()


+---+----------+----------+-------+----+----------------+------------+-------------+--------------------+------------------+-----------+------------------------------+
| id|product_id|created_at|sold_at|cost|product_category|product_name|product_brand|product_retail_price|product_department|product_sku|product_distribution_center_id|
+---+----------+----------+-------+----+----------------+------------+-------------+--------------------+------------------+-----------+------------------------------+
|  0|         0|         0|  26775|   0|               0|           0|           37|                   0|                 0|          0|                             0|
+---+----------+----------+-------+----+----------------+------------+-------------+--------------------+------------------+-----------+------------------------------+



- Terdapat missing value yang tinggi pada kolom sold at namun tidak dapat dihapus karena memberi petunjuk bahwa barang masih berada di gudang dan kapan barang keluar, serta sedikit mising value pada produk brand yang akan dihapus karena berisi data yang berulang dari table produk.

In [35]:
df_inv = df_inv.na.fill({"product_brand": "Unknown"})
df_inv.show()

+------+----------+-------------------+-------------------+------------------+----------------+--------------------+-------------------+--------------------+------------------+--------------------+------------------------------+
|    id|product_id|         created_at|            sold_at|              cost|product_category|        product_name|      product_brand|product_retail_price|product_department|         product_sku|product_distribution_center_id|
+------+----------+-------------------+-------------------+------------------+----------------+--------------------+-------------------+--------------------+------------------+--------------------+------------------------------+
| 82147|     28953|2023-06-18 21:17:41|2023-08-16 13:49:41| 7.429869883489907|     Accessories|100% Silk Woven G...|          TheTieBar|  17.989999771118164|               Men|7B889DA86FA368B08...|                             7|
| 77591|     13944|2024-08-08 20:15:11|2024-09-14 14:03:11|1.7914200385826824|     A

### Data transform dan cleaning tabel entity Distribution Center

In [36]:
df_dist = spark.read.csv('/Users/mac/Documents/Hacktiv8/GC6/p2-coda001-rmt-gc6-mirzasjarief/look_distribution_centers.csv', header=True, inferSchema=True)
df_dist.show()

+---+--------------------+--------+---------+------------------------+
| id|                name|latitude|longitude|distribution_center_geom|
+---+--------------------+--------+---------+------------------------+
| 10|         Savannah GA| 32.0167| -81.1167|    POINT(-81.1167 32...|
|  4|      Los Angeles CA|   34.05|  -118.25|    POINT(-118.25 34.05)|
|  3|          Houston TX| 29.7604| -95.3698|    POINT(-95.3698 29...|
|  7|     Philadelphia PA|   39.95| -75.1667|    POINT(-75.1667 39...|
|  1|          Memphis TN| 35.1174| -89.9711|    POINT(-89.9711 35...|
|  5|      New Orleans LA|   29.95| -90.0667|    POINT(-90.0667 29...|
|  9|       Charleston SC| 32.7833| -79.9333|    POINT(-79.9333 32...|
|  6|Port Authority of...|  40.634| -73.7834|    POINT(-73.7834 40...|
|  8|           Mobile AL| 30.6944| -88.0431|    POINT(-88.0431 30...|
|  2|          Chicago IL| 41.8369| -87.6847|    POINT(-87.6847 41...|
+---+--------------------+--------+---------+------------------------+



In [37]:
df_dist = df_dist.drop("latitude","longitude")

df_dist.show()

+---+--------------------+------------------------+
| id|                name|distribution_center_geom|
+---+--------------------+------------------------+
| 10|         Savannah GA|    POINT(-81.1167 32...|
|  4|      Los Angeles CA|    POINT(-118.25 34.05)|
|  3|          Houston TX|    POINT(-95.3698 29...|
|  7|     Philadelphia PA|    POINT(-75.1667 39...|
|  1|          Memphis TN|    POINT(-89.9711 35...|
|  5|      New Orleans LA|    POINT(-90.0667 29...|
|  9|       Charleston SC|    POINT(-79.9333 32...|
|  6|Port Authority of...|    POINT(-73.7834 40...|
|  8|           Mobile AL|    POINT(-88.0431 30...|
|  2|          Chicago IL|    POINT(-87.6847 41...|
+---+--------------------+------------------------+



- Untuk efisiensi langkah berikutnya menghapus kolo longitude dan langitude karena sudah diwakilkan dengan distribution_center_geom

### Connecting to Database (postgresSQL)

In [38]:
# Connect with database credentials
db_user = "postgres"
db_password = "postgres"
db_host = "localhost"
db_port = "5432"
db_name = "the_look"

# Create a connection string
connection_string = f"postgresql+psycopg2://{'postgres'}:{'postgres'}@{'localhost'}:{5432}/{'the_look'}"

# Create the database engine
engine = create_engine(connection_string)

# transform to pandas and load to SQL
df_users = df_users.toPandas()
df_users.to_sql("customer_data", engine, if_exists='replace', index=False)

184

- Mengisi data base credentials load customer_data ke database

In [39]:
# Create a connection string
connection_string = f"postgresql+psycopg2://{'postgres'}:{'postgres'}@{'localhost'}:{5432}/{'the_look'}"

# Create the database engine
engine = create_engine(connection_string)

# transform to pandas and load to SQL
df_products = df_products.toPandas()
df_products.to_sql("products_data", engine, if_exists='replace', index=False)

120

- load products_data ke database

In [40]:
# Create a connection string
connection_string = f"postgresql+psycopg2://{'postgres'}:{'postgres'}@{'localhost'}:{5432}/{'the_look'}"

# Create the database engine
engine = create_engine(connection_string)

# transform to pandas and load to SQL
df_orders_date = df_orders_date.toPandas()
df_orders_date.to_sql("orders_data", engine, if_exists='replace', index=False)

905

- load orders data

In [41]:
# Create a connection string
connection_string = f"postgresql+psycopg2://{'postgres'}:{'postgres'}@{'localhost'}:{5432}/{'the_look'}"

# Create the database engine
engine = create_engine(connection_string)

# transform to pandas and load to SQL
df_inv = df_inv.toPandas()
df_inv.to_sql("inventory_data", engine, if_exists='replace', index=False)

571

- load inventory_data ke database

In [42]:
# Create a connection string
connection_string = f"postgresql+psycopg2://{'postgres'}:{'postgres'}@{'localhost'}:{5432}/{'the_look'}"

# Create the database engine
engine = create_engine(connection_string)

# transform to pandas and load to SQL
df_dist = df_dist.toPandas()
df_dist.to_sql("distribution_data", engine, if_exists='replace', index=False)

10

- load distribution_data ke database

In [43]:
# Create a connection string
connection_string = f"postgresql+psycopg2://{'postgres'}:{'postgres'}@{'localhost'}:{5432}/{'the_look'}"

# Create the database engine
engine = create_engine(connection_string)

# transform to pandas and load to SQL
df_order_items_date= df_order_items_date.toPandas()
df_order_items_date.to_sql("order_items_data", engine, if_exists='replace', index=False)

936

- load order_items_data ke database

Google Slide Link :
https://docs.google.com/presentation/d/1mBx0e1DD5zaB4deAzdCkDpsYNUme-ippkzZscIUPx3Y/edit?usp=sharing